[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/Arif-PhyChem/trace_conservation/blob/main/physics_agnostic.ipynb)

In [1]:
import sys
import os

! git clone https://github.com/Arif-PhyChem/MLQD.git

Cloning into 'MLQD'...
remote: Enumerating objects: 1008, done.
remote: Counting objects: 100% (284/284), done.
remote: Compressing objects: 100% (190/190), done.
remote: Total 1008 (delta 135), reused 199 (delta 93), pack-reused 724
Receiving objects: 100% (1008/1008), 34.97 MiB | 11.68 MiB/s, done.
Resolving deltas: 100% (427/427), done.


In [2]:
# install the required packages

! pip install tensorflow --upgrade
! pip install sklearn
! pip install hyperopt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 32.6 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Fo

In [39]:
# Add MLQD's path to system path

mlqd_dir='MLQD/dev_ver'
sys.path.append(mlqd_dir)

from evolution import quant_dyn



In [40]:
# get training and test data files from github
! git clone https://github.com/Arif-PhyChem/trace_conservation.git

fatal: destination path 'trace_conservation' already exists and is not an empty directory.


# Physics agnostic model for spin boson (SB) model

In the context of physics-agnostic neural networks, we train separate Convolutional Neural Networks (CNNs) for each element of the diagonal Reduced Density Matrix (RDM). To facilitate this process, we offer pre-prepared training files named 'x.npy' and 'y.npy'. However, if you wish to create your own training files, you can utilize the MLQD tool, available on GitHub at https://github.com/Arif-PhyChem/MLQD, in conjunction with the SB dataset from the QD3SET-1 database, which can be accessed via the DOI link https://doi.org/10.25452/figshare.plus.c.6389553

In [ ]:
# sb training files (x and y) for state 1 and state 2
! ls trace_conservation/training_data/sb/physics_agnostic/state_1
! ls trace_conservation/training_data/sb/physics_agnostic/state_2

x.npy  y.npy
x.npy  y.npy


Below we train a CNN model for state 1 and state 2 independently. If you prefer not to train your own CNN model, you can opt to use the pre-trained models we have provided (scroll down below).




## Training CNN model for State-1 with MLQD

---



In [10]:
# input for MLQD package, more details are given at https://github.com/Arif-PhyChem/MLQD
param={
    'n_states': 2,
      'QDmodel': 'createQDmodel',     # str: create QD model. The dafault option is useQDmodel
      'QDmodelType': 'RCDYN',           # str: The type of model. Here KRR and the default option is OSTL
      'prepInput' : 'False',           # str: Prepare input files from the data (Default 'False')
      'XfileIn': 'trace_conservation/training_data/sb/physics_agnostic/state_1/x',  # str: (Optional) The prepared X file
      'YfileIn': 'trace_conservation/training_data/sb/physics_agnostic/state_1/y',  # str: (Optional) The prepared Y file
      'hyperParam': 'False',  # str: Default is 'False', we can pass 'True' (optimize the hyperparameters) or 'False' (don't optimize and run with the default values)
      'patience': 30,
      'OptEpochs': 30,
      'TrEpochs': 10,
      'max_evals': 30,
      'systemType': 'SB',
      'QDmodelOut': 'sb_model_4_state_1'    # str: (Optional), providing a name to save the model at
      }
quant_dyn(**param)



MLQD is a python package developed for Machine Learning-based Quantum Dissipative Dynamics,
 				  Version 1.1.1
			 https://github.com/Arif-PhyChem/MLQD

 			 Copyright (c) 2023 Arif Ullah

All rights reserved. This work is licensed under the Apache Software License 2.0


The above copyright notice and this permission notice shall be included 
in all copies or substantial portions of the Software.


The software is provided "as is" without warranty of any kind, express or implied, 
including but not limited to the warranties ofmerchantability, fitness for a particular 
purpose and noninfringement. In no event shall the authors or copyright holders be 
liable for any claim, damages or other liability, whether in an action of contract, tort or otherwise, 
arising from, out of or in connection with the software or the use or other dealings in the software.


				 Cite as:

1) Ullah A. and Dral P. O., Computer Physics Communications, 2023, 294, 108940
2) Ullah A. and Dral P. O., New Journ

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
2000/2000 - 25s - 12ms/step - loss: 7.7301e-04 - val_loss: 1.5335e-05
Epoch 2/10
2000/2000 - 40s - 20ms/step - loss: 3.6076e-05 - val_loss: 7.1952e-06
Epoch 3/10
2000/2000 - 41s - 21ms/step - loss: 3.0469e-05 - val_loss: 2.3430e-05
Epoch 4/10
2000/2000 - 40s - 20ms/step - loss: 2.2564e-05 - val_loss: 3.8969e-06
Epoch 5/10
2000/2000 - 40s - 20ms/step - loss: 1.9450e-05 - val_loss: 3.7838e-06
Epoch 6/10
2000/2000 - 21s - 10ms/step - loss: 1.4409e-05 - val_loss: 4.0479e-06
Epoch 7/10
2000/2000 - 20s - 10ms/step - loss: 7.1651e-06 - val_loss: 1.2959e-06
Epoch 8/10
2000/2000 - 21s - 10ms/step - loss: 6.2882e-06 - val_loss: 2.0833e-06
Epoch 9/10
2000/2000 - 20s - 10ms/step - loss: 4.7819e-06 - val_loss: 2.6857e-06
Epoch 10/10
2000/2000 - 20s - 10ms/step - loss: 4.2141e-06 - val_loss: 4.1749e-06
Restoring model weights from the end of the best epoch: 7.
cnn.OSTL_default: OSTL model is saved as " sb_model_4_state_1.keras "
Train_ml.RCDYN: Time taken for training = 288.43226623535156

## Training CNN model for State-2 with MLQD

In [12]:
# input for MLQD package, more details are given at https://github.com/Arif-PhyChem/MLQD
param={
    'n_states': 2,
      'QDmodel': 'createQDmodel',     # str: create QD model. The dafault option is useQDmodel
      'QDmodelType': 'RCDYN',           # str: The type of model. Here KRR and the default option is OSTL
      'prepInput' : 'False',           # str: Prepare input files from the data (Default 'False')
      'XfileIn': 'trace_conservation/training_data/sb/physics_agnostic/state_2/x',           # str: (Optional) The prepared X file
      'YfileIn': 'trace_conservation/training_data/sb/physics_agnostic/state_2/y',           # str: (Optional) The prepared Y file
      'hyperParam': 'False',           # str: Default is 'False', we can pass 'True' (optimize the hyperparameters) or 'False' (don't optimize and run with the default values)
      'patience': 30,
      'OptEpochs': 30,
      'TrEpochs': 10,
      'max_evals': 30,
      'systemType': 'SB',
      'QDmodelOut': 'sb_model_4_state_2'    # str: (Optional), providing a name to save the model at
      }
quant_dyn(**param)


MLQD is a python package developed for Machine Learning-based Quantum Dissipative Dynamics,
 				  Version 1.1.1
			 https://github.com/Arif-PhyChem/MLQD

 			 Copyright (c) 2023 Arif Ullah

All rights reserved. This work is licensed under the Apache Software License 2.0


The above copyright notice and this permission notice shall be included 
in all copies or substantial portions of the Software.


The software is provided "as is" without warranty of any kind, express or implied, 
including but not limited to the warranties ofmerchantability, fitness for a particular 
purpose and noninfringement. In no event shall the authors or copyright holders be 
liable for any claim, damages or other liability, whether in an action of contract, tort or otherwise, 
arising from, out of or in connection with the software or the use or other dealings in the software.


				 Cite as:

1) Ullah A. and Dral P. O., Computer Physics Communications, 2023, 294, 108940
2) Ullah A. and Dral P. O., New Journ

## Dynamics prediction for a test trajectory.
For this task, we are leveraging the pre-trained CNN models that we have made available. However, if you have trained your own CNN models as previously described, you have the option to employ those models instead. This flexibility allows you to choose between our provided models and your custom-trained ones, depending on your specific requirements or preferences.  

In [18]:
# Trained CNN models we provide
! ls trace_conservation/trained_models/sb/physics_agnostic/state_1/
! ls trace_conservation/trained_models/sb/physics_agnostic/state_2/
# test tarjectory
! ls trace_conservation/test_data/sb/
! ls

RC_SB_model.keras
RC_SB_model.keras
2_epsilon-0.0_Delta-1.0_lambda-0.2_gamma-10.0_beta-0.75.npy
MLQD  sample_data  sb_model_4_state_1.keras  sb_model_4_state_2.keras  trace_conservation


In [25]:
import numpy as np
import tensorflow as tf

xx = np.load('trace_conservation/test_data/sb/2_epsilon-0.0_Delta-1.0_lambda-0.2_gamma-10.0_beta-0.75.npy')


n_states = 2
xlength = 41
time = 10
time_step = 0.05
time_range=0
tt = time_range

# time range
for i in range(0, xlength + int(time/time_step)-1):
    tt += time_step
    time_range = np.append(time_range, tt)


# trained CNN models
model_1 = tf.keras.models.load_model('sb_model_4_state_1.keras', compile=False) #('trace_conservation/trained_models/sb/physics_agnostic/state_1/RC_SB_model.keras') # state 1 CNN model
model_2 = tf.keras.models.load_model('sb_model_4_state_2.keras', compile=False) #('trace_conservation/trained_models/sb/physics_agnostic/state_2/RC_SB_model.keras') # state 2 CNN model

models = [model_1, model_2]

x = np.zeros((xlength, n_states), dtype=float)
y = np.zeros((len(time_range), n_states), dtype=float)
yhat = np.zeros((1, n_states), dtype=float)

trace = np.zeros((len(time_range), 1), dtype=float)
labels = [1, 4]

# short time dynamics for input
i = 0
for label in labels:
    x[:,i] = np.real(xx[0:xlength, label])
    i += 1
y[0:xlength, :] = x[:,:]

print(y)

# trace of the input dynamics
for i in range (0, xlength):
  tr = 0
  for j in labels:
    tr += np.real(xx[i, j])
  trace[i, 0] = tr

print(trace)

print('****************************************************************')
print('Predicting dynamics')
print('****************************************************************')

for i in range(xlength, len(time_range)):
    l =0
    for j in range (0, n_states):
        x_pred = x[:, j]
        x_pred = x_pred.reshape(1, x.shape[0], 1) # reshape the input
        yhat[0, j] =  models[l].predict(x_pred, verbose=0)[0][0]
        l += 1

    #
    trace[i, 0] = np.sum(yhat)
    print(trace[i,0])
    x = np.delete(x, 0, 0)
    x = np.r_[x, yhat]
    y[i, :] = yhat[0, :]
np.save('tc_traj', np.c_['-1', time_range[:], y[:]])
print('Dynamics with no trace conservation is generated and saved in a file with name tc_traj.npy')

[[1.         0.        ]
 [0.99751179 0.00248821]
 [0.99012173 0.00987827]
 [0.97798798 0.02201202]
 [0.96131528 0.03868472]
 [0.94034549 0.05965451]
 [0.91535324 0.08464676]
 [0.88664318 0.11335682]
 [0.85453201 0.14546799]
 [0.8194022  0.1805978 ]
 [0.78162156 0.21837844]
 [0.74159207 0.25840793]
 [0.69973481 0.30026519]
 [0.65646874 0.34353126]
 [0.61223473 0.38776527]
 [0.56745937 0.43254063]
 [0.52258638 0.47741362]
 [0.47804442 0.52195558]
 [0.43425755 0.56574245]
 [0.39163615 0.60836385]
 [0.35057777 0.64942223]
 [0.31145206 0.68854794]
 [0.27461258 0.72538742]
 [0.24039247 0.75960753]
 [0.20907181 0.79092819]
 [0.18091836 0.81908164]
 [0.15617548 0.84382452]
 [0.13501437 0.86498563]
 [0.11763596 0.88236404]
 [0.1041336  0.8958664 ]
 [0.09458467 0.90541533]
 [0.08904005 0.91095995]
 [0.08750443 0.91249557]
 [0.08993894 0.91006106]
 [0.09626696 0.90373304]
 [0.10638424 0.89361576]
 [0.12015282 0.87984718]
 [0.1373963  0.8626037 ]
 [0.15790712 0.84209288]
 [0.18144891 0.81855109]
